In [1]:
import cv2
import numpy as np

def segmentation(image):
    # Convert the image to grayscale 
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    #Apply segmentation (example: thresholding) 
    _, segmented_image = cv2.threshold(gray_image, 127, 255, cv2.THRESH_BINARY)
    
    return segmented_image
    
    
def cannyedge(image):
    """
    Return the canny edge using the auto feature
    Measure the low and high and tolerance that lines would
    show in the image

    Parameters
    ----------
    image : TYPE
        DESCRIPTION.

    Returns
    -------
    auto_canny : TYPE
        DESCRIPTION.

    """
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Tolerance value for lines
    sigma = 0.3 
    median = np.median(image) 
    
    
    # Automatic canny edge detection using computed median
    # 30% low in this case
    lower = int(max(0, (1.0 - sigma) * median))
    
    # Lower the the threshold sigma % lower than median
    # if the value is below 0 then take 0 as the value

    upper = int(min(255, (1.0 + sigma) * median))
    auto_canny = cv2.Canny(image, lower, upper)
    
    # cv2.imshow("AutoCanny", auto_canny)
    # cv2.waitKey(0)
    # cv2.destroyWindow()
    return auto_canny

def grayscale(image):
    """
    Will write an grayscale image in the specified path

    Parameters
    ----------
    image : Numpy.array
        A valid image file
    path : String
        Path of file

    Returns
    -------
    None.

    """
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

if __name__ == "__main__":
    pass
      
# Future preprocess refined for image lines extraction
def convolution2d(image):
    """  
    
    Laplacian of Gaussian (LoG) kernel
    
    Parameters
    ----------
    image : Numpy array
        A valid image

    Returns
    -------
    out : Numpy array
        The result of filtering the image using the Laplacian of Gaussian
        (LoG) kernel

    """
    

    kernel = np.array([[0, 1, 0],
                    [1, -4, 1],
                    [0, 1, 0]])
    out = cv2.filter2D(image, ddepth=-1, kernel=kernel)
    out = cv2.convertScaleAbs(out)
    
    return out



In [2]:
import os
import zipfile
import matplotlib.pyplot as plt

def visualize(images, labels, rows=1, cols=1):
    """
    Visualize an image along with the given string of labels
    It will only accept a list of images that is compatible
    with the matplotlib.pyplot 

    Parameters
    ----------
    images : List[]
        Accept image of list that is compatible with matplotlib.pyplot
    labels : List[]
        Accept a l ist of string
    rows : Int, optional
        The row of subplots that will be displayed. The default is 1.
    cols : TYPE, optional
        The column of subplots that will be displayed. The default is 1.

    Returns
    -------
    None.

    """
    _, axs = plt.subplots(nrows=rows, ncols=cols)
    for pic, ax, label in zip(images, axs.flatten(), labels):
        ax.imshow(pic)
        ax.set_title(label)
        ax.axis('off')
    
    plt.savefig("default.png", dpi=150)
    
def create_dataset(src):
    """
    Takes an source path that will create a dataset assuming it has
    'labels' (subfolder) and  storing 'paths' in the dictionary

    Parameters
    ----------
    src : STRING
        Path of the directory to be read.

    Returns
    -------
    data : DICTIONARY
          Returns ['labels', 'paths'] of the data read from the source passed.

    """
    data = dict()
    data['labels'] = [] 
    #data['images'] = []
    data['paths'] = []

    # Read the specified  datasets of images in each subfolder with
    # their respective labels
    for root, subdir, files in os.walk(src):
        for class_name in subdir: 
            class_dir = os.path.join(root, class_name) 
            for filename in os.listdir(class_dir): 
                img_path = os.path.join(class_dir, filename)
                #with Image.open(img_path) as img:
                data['labels'].append(class_name)
                data['paths'].append(img_path)   
    return data
#############################
#           Folders 
#
############################

def create_folders(src, labels):
    os.makedirs(src, exist_ok=True)
    for i in labels:
        out_path = os.path.join(src, i)
        os.makedirs(out_path, exist_ok=True)
        
def create_folders_for_labels(output_dir, dataset_labels, extra_labels):
    """
    Creates folders for each label and each extra label inside their respective class folders.

    Parameters:
        output_dir (str): The main output directory.
        dataset_labels (list): List of labels.
        extra_labels (list): List of extra labels.

    Returns:
        None
    """
    for label in dataset_labels:
        label_folder = os.path.join(output_dir, label)
        create_folders(label_folder, extra_labels)
        
def zip_folder(folder_path, zip_path):
    """
    Compresses the entire folder at the specified path into a ZIP file.

    Parameters:
        folder_path (str): Path to the folder to be zipped.
        zip_path (str): Path to save the ZIP file.

    Returns:
        None
    """
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                zipf.write(file_path, os.path.relpath(file_path, folder_path))        

def extract_zip(zip_path, output_path):
    """
    Extract the entire contents of zip file at the specified output path

    Parameters
    ----------
    zip_path : TYPE
        DESCRIPTION.
    output_path : TYPE
        DESCRIPTION.

    Returns
    -------
    None.

    """
    with zipfile.ZipFile(zip_path, 'r') as zip:
        # Print the contents of zip
        zip.printdir()
        print('Extracting the contents...')
        zip.extractall(output_path)
        print('Done!')
        

In [3]:
# -*- coding: utf-8 -*-

path = r".\tree"
zpath = r".\FAITHBark.zip"
extract_zip(zpath, '.')

dataset = create_dataset(path)
IMG_SIZE = (223, 223)

output_dir = "output"
create_folders(output_dir, dataset['labels'])

extra_labels = ['grayscale', 'segment', 'cannyedge', 'convolution2d']

create_folders_for_labels(output_dir, dataset['labels'], extra_labels)

for path, subf in zip(dataset['paths'], dataset['labels']):
    
    image = cv2.imread(path)

    if image.shape != IMG_SIZE: # Verify the image is uniformed
        image = cv2.resize(image, IMG_SIZE)
        
    # Slice the string for filename
    start = path.rfind('\\')  # Windows path separator
    end = path.rfind ('.')  # Before the extension
    
    filename = path[start + 1 : end] + ".jpg"
    
    
    # Apply preprocessing to the image and save in corresponding subfolder
    for xlabel in extra_labels:
        output_subfolder = os.path.join(output_dir, subf, xlabel, filename)
        
        if xlabel == 'grayscale':
            processed_image = grayscale(image)
            cv2.imwrite(output_subfolder, processed_image)
        elif xlabel == 'segment':
            processed_image = segmentation(image)
            cv2.imwrite(output_subfolder, processed_image)
        elif xlabel == 'cannyedge':
            processed_image = cannyedge(image)
            cv2.imwrite(output_subfolder, processed_image)
        elif xlabel == 'convolution2d':
            processed_image = convolution2d(image)
            cv2.imwrite(output_subfolder, processed_image)
            
        print(f"Saved preprocess image {output_subfolder}")   
    
print(f"Preprocessed images created and saved in {output_dir}")
zip_folder(output_dir, 'output.zip')

ModuleNotFoundError: No module named 'utils'